# Apache Spark RDDs Einführung

In [1]:
from pyspark import SparkContext
sc = SparkContext("local", "Simple App")

sc

<SparkContext master=local appName=Simple App>

In [ ]:
# Original post and context: 
# http://www.supergloo.com/fieldnotes/apache-spark-quick-start-with-python-new-york-city-uber-trips


## Datei einlesen

In [2]:
ut = sc.textFile("data/Uber-Jan-Feb-FOIL.csv")

## Aktionen

In [3]:
ut.count()

355

In [4]:
ut.first()

'dispatching_base_number,date,active_vehicles,trips'

## Transformationen

In [5]:
rows = ut.map(lambda line: line.split(","))

In [6]:
rows.map(lambda row: row[0]) \
    .distinct() \
    .count()

7

In [7]:
rows.map(lambda row: row[0]) \
    .distinct() \
    .collect()

['B02617',
 'B02764',
 'B02682',
 'B02765',
 'B02598',
 'B02512',
 'dispatching_base_number']

In [ ]:
rows.filter(lambda row: "B02617" in row).count()

In [26]:
base02617 = rows.filter(lambda row: "B02617" in row)

In [27]:
base02617.filter(lambda row: int(row[3]) > 15000).count()

6

In [28]:
base02617.filter(lambda row: int(row[3]) > 15000).map(lambda day: day[1]).distinct().count()

6

In [30]:
filteredRows = sc.textFile("data/Uber-Jan-Feb-FOIL.csv") \
    .filter(lambda line: "base" not in line) \
    .map(lambda line:line.split(","))

filteredRows.map(lambda kp: (kp[0], int(kp[3]))) \
    .reduceByKey(lambda k,v: k + v) \
    .collect()

[('B02617', 725025),
 ('B02764', 1914449),
 ('B02682', 662509),
 ('B02765', 193670),
 ('B02598', 540791),
 ('B02512', 93786)]

In [31]:
filteredRows.map(lambda kp: (kp[0], int(kp[3])) ) \
    .reduceByKey(lambda k,v: k + v) \
    .takeOrdered(10, key=lambda x: -x[1])

[('B02764', 1914449),
 ('B02617', 725025),
 ('B02682', 662509),
 ('B02598', 540791),
 ('B02765', 193670),
 ('B02512', 93786)]